In [ ]:
%pip install google-api-python-client -q

from googleapiclient import discovery
import json
from dotenv import load_dotenv
import os

# load enviornment variables
load_dotenv()

API_KEY = os.getenv("GOOGLE_API_KEY")

# creates connection to google's perspective api
client = discovery.build(
    "commentanalyzer", "v1alpha1",
    developerKey=API_KEY,
    discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
    static_discovery=False,
)

# defines what text to analyze ("I hate Python...")
# requests toxicity score (how offensive/harmful the text is)
analyze_request = {
    'comment': {'text': 'I hate Python programming. It sucks big time.'},
    'requestedAttributes': {'TOXICITY': {}}
}

# send request and get score
response = client.comments().analyze(body=analyze_request).execute()
score = response['attributeScores']['TOXICITY']['summaryScore']['value']
print("Toxicity Score: ", score)

# Protected Attribute and Creating Examples

I will be choosing the protected attribute of Gender. Next, I'll manually create 20 examples (10 toxic, 10 non-toxic) for each gender (Male and Female), resulting in 40 examples total. This will be stored in the CSV file data.csv.

In [ ]:
# using gemini to generate toxicity judgement

import google.generativeai as genai
import time

genai.configure(api_key=)